In [1]:
%load_ext rustdef

load rustdef


<IPython.core.display.Javascript object>

In [2]:
%rustdef depends --help

usage: %rustdef depends [-h] crates [crates ...]

Add crate dependencies

positional arguments:
  crates      Dependencies to be added

optional arguments:
  -h, --help  show this help message and exit


In [3]:
%rustdef depends ndarray

In [4]:
%%rustdef
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    println!("{}", a);
    Ok(z.into())
}

Use previous build


In [5]:
import numpy as np

In [6]:
axpy(2.4, np.array([1.0, 3.2, 2.9]), np.array([2.3, 10.9, -2]))

2.4


array([ 4.7 , 18.58,  4.96])

In [7]:
%%rustdef
use rustdef::numpy::Array;

// immutable example
#[pyfunction]
fn axpy<'a>(a: f64, x: Array<'a, f64>, y: Array<'a, f64>) -> PyResult<Array<'a, f64>> {
    let x = x.to_array();
    let y = y.to_array();
    let z = a * x + y;
    Ok(z.into())
}

// mutable example
#[pyfunction]
fn mult_inline<'a>(a: f64, x: Array<'a, f64>) -> PyResult<()> {
    let mut x = x.into_mut_array()?;
    x *= a;
    Ok(())
}

Use previous build


In [8]:
x = np.random.rand(10)
y = np.random.rand(10)
x, y

(array([0.92143706, 0.00225896, 0.10215355, 0.40368477, 0.04606151,
        0.50350768, 0.37160328, 0.11971092, 0.05561732, 0.53525992]),
 array([0.41618282, 0.79195973, 0.8778752 , 0.6719745 , 0.7536344 ,
        0.98451322, 0.28129915, 0.98359887, 0.11583479, 0.22774309]))

In [9]:
axpy(3.0, x, y)

array([3.18049401, 0.79873661, 1.18433586, 1.8830288 , 0.89181892,
       2.49503626, 1.39610898, 1.34273164, 0.28268675, 1.83352285])

In [10]:
mult_inline(12.0, x)

In [11]:
x

array([11.05724474,  0.02710752,  1.22584263,  4.84421718,  0.55273809,
        6.04209218,  4.45923932,  1.43653107,  0.66740786,  6.42311904])